## LSTM на оригинальном датасете

Попытка сделать монофонический выход из сетки

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

Сделаем также пользовательский импорт

In [2]:
from decode_patterns import data_conversion

Загружаем датасет

In [3]:
# import dataset
drum, bass = data_conversion.make_lstm_dataset(height=16, patterns_file="decode_patterns/train.tsv", mono=True)


# define shuffling of dataset
def shuffle(A, B, p=0.8):
    # take 80% to training, other to testing
    L = len(A)
    idx = np.arange(L) < p*L
    np.random.shuffle(idx)
    yield A[idx]
    yield B[idx]
    yield A[np.logical_not(idx)]
    yield B[np.logical_not(idx)]
    
# we can shuffle train and test set like this:
drum_train, bass_train, drum_test, bass_test = shuffle(drum, bass)

# selecting a validation set
drum_validation, bass_validation = data_conversion.make_lstm_dataset(height=16
                                                        , patterns_file="decode_patterns/validation.tsv"
                                                        , mono=True)
drum_validation = torch.tensor(drum_validation, dtype=torch.float)
bass_validation = torch.tensor(bass_validation, dtype=torch.float)
    

In [4]:
drum_train.shape

(40000, 16, 14)

Модель определим в самом простом варианте, который только можно себе представить -- как в примере с конечным автоматом

In [5]:
# попробуем определить модель LSTM как конечный автомат
class DrumNBassLSTM(nn.Module):
    def __init__(self):
        super(DrumNBassLSTM, self).__init__()
        # one input neuron, one output neuron, one layer in LSTM block
        self.input_size = 14
        self.hidden_size = 34
        self.layer_count = 1
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.layer_count)
        self.embed_layer = nn.Linear(self.hidden_size, 1)
        self.sigm = nn.Sigmoid()
    
    def forward(self, input):
        # пусть в input у нас приходит вектор размерности (64, 32, 14)
        # то есть 64 отсчёта, тридцать два примера (минибатч), 14 значение в каждом (барабанная партия)
        output, _ = self.lstm(input)
        output = self.sigm(self.embed_layer(output))*37
        return output

In [6]:
# часть обучения
dnb_lstm = DrumNBassLSTM()

criterion = nn.MSELoss()

# для сравнения моделей необходим reconstruction_loss
reconstruction_loss = criterion

# оценим также и разнообразие мелодии по её.. дисперсии?)
# def melody_variety(melody):
#     return 1/(1 + (melody.sum(axis=2) > 1).int())
    
# criterion = nn.NLLLoss() # -- этот товарищ требует, чтобы LSTM выдавал классы,
# criterion = nn.CrossEntropyLoss() # и этот тоже
# (числа от 0 до C-1), но как всё-таки его заставить это делать?...
# optimizer = optim.SGD(dnb_lstm.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(dnb_lstm.parameters(), lr=0.001)

Найденные баги и их решения:

https://stackoverflow.com/questions/56741087/how-to-fix-runtimeerror-expected-object-of-scalar-type-float-but-got-scalar-typ

https://stackoverflow.com/questions/49206550/pytorch-error-multi-target-not-supported-in-crossentropyloss/49209628

https://stackoverflow.com/questions/56243672/expected-target-size-50-88-got-torch-size50-288-88

In [7]:
epoch_count = 72
batch_size = 32
shuffle_every_epoch = True
    
if shuffle_every_epoch:
    print(f"shuffle_every_epoch is on")
else:
    print(f"shuffle_every_epoch is off")
    # shuffle train and test set:
    drum_train, bass_train, drum_test, bass_test = shuffle(drum, bass)
    drum_train = torch.tensor(drum_train, dtype=torch.float)
    bass_train = torch.tensor(bass_train, dtype=torch.float)
    drum_test = torch.tensor(drum_test, dtype=torch.float)
    drum_test = torch.tensor(drum_test, dtype=torch.float)
        
for epoch in range(epoch_count):  # loop over the dataset multiple times
    print(f"Epoch #{epoch}")
    if shuffle_every_epoch:
        # shuffle train and test set:
        drum_train, bass_train, drum_test, bass_test = shuffle(drum, bass)
        drum_train = torch.tensor(drum_train, dtype=torch.float)
        bass_train = torch.tensor(bass_train, dtype=torch.float)
        drum_test = torch.tensor(drum_test, dtype=torch.float)
        bass_test = torch.tensor(bass_test, dtype=torch.float)
        
    examples_count = drum_train.size()[0]
    examples_id = 0
    
    running_loss = 0.0
    runnint_count = 0
    batch_id = 0
    while examples_id < examples_count:
        batch_drum_train = drum_train[examples_id:examples_id + batch_size,:,:].transpose(0,1)
        batch_bass_train = bass_train[examples_id:examples_id + batch_size,].transpose(0,1)
        # transpose нужен для обмена размерности батча и размерности шагов
#         print(f"batch_drum_train:{batch_drum_train.size()}, batch_bass_train:{batch_bass_train.size()}")

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        bass_outputs = dnb_lstm(batch_drum_train).squeeze()
        
        # loss = criterion(bass_outputs, batch_bass_train.long())
        loss = criterion(bass_outputs, batch_bass_train)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        runnint_count += 1
        period = 5
        if batch_id % period == 0 or examples_id + batch_size >= examples_count:
            print('[%d, %5d] train loss: %.7f' %
                  (epoch + 1, batch_id + 1, running_loss / runnint_count))
            running_loss = 0.0
            runnint_count = 1
            
        # update batch info
        examples_id += batch_size
        batch_id += 1
        
    # here we can insert measure error on test set
    with torch.no_grad():
        bass_outputs = dnb_lstm(drum_test).squeeze()
        
        test_count = len(drum_test)
        test_loss = 0
        for k in range(test_count):
            test_loss += reconstruction_loss(bass_outputs[k], bass_test[k])
        print(f"#{epoch + 1} reconstruction test loss: {test_loss/test_count}")
    

#should check accuracy on validation set
with torch.no_grad():
    bass_outputs = dnb_lstm(drum_validation).squeeze()

    validation_count = len(drum_validation)
    validation_loss = 0
    for k in range(validation_count):
        validation_loss += reconstruction_loss(bass_outputs[k], bass_validation[k])
    print(f"#{epoch + 1} reconstruction validation loss: {validation_loss/validation_count}")
print('Finished Training')

shuffle_every_epoch is on
Epoch #0
[1,     1] train loss: 221.3494110
[1,     6] train loss: 187.6525447
[1,    11] train loss: 172.0311991
[1,    16] train loss: 167.4183121
[1,    21] train loss: 154.1088537
[1,    26] train loss: 143.8433736
[1,    31] train loss: 134.0485382
[1,    36] train loss: 121.4743195
[1,    41] train loss: 88.4782550
[1,    46] train loss: 76.1715482
[1,    51] train loss: 63.7691358
[1,    56] train loss: 57.8340842
[1,    61] train loss: 63.8155594
[1,    66] train loss: 56.5195491
[1,    71] train loss: 59.0157026
[1,    76] train loss: 60.7472343
[1,    81] train loss: 55.4998086
[1,    86] train loss: 55.2104429
[1,    91] train loss: 57.7542299
[1,    96] train loss: 53.9977366
[1,   101] train loss: 56.6058102
[1,   106] train loss: 61.4204451
[1,   111] train loss: 58.8627205
[1,   116] train loss: 52.9986134
[1,   121] train loss: 53.3456720
[1,   126] train loss: 53.9071045
[1,   131] train loss: 55.1609535
[1,   136] train loss: 56.5547638
[1,  

[1,  1196] train loss: 46.8829772
[1,  1201] train loss: 47.3687553
[1,  1206] train loss: 47.0243950
[1,  1211] train loss: 39.7245483
[1,  1216] train loss: 49.6931915
[1,  1221] train loss: 45.1132145
[1,  1226] train loss: 51.1668269
[1,  1231] train loss: 40.4735349
[1,  1236] train loss: 52.4269441
[1,  1241] train loss: 38.3479468
[1,  1246] train loss: 49.1155637
[1,  1250] train loss: 40.9427193
#1 reconstruction test loss: 53.360755920410156
Epoch #1
[2,     1] train loss: 56.0025597
[2,     6] train loss: 42.0231991
[2,    11] train loss: 50.2679965
[2,    16] train loss: 47.8747298
[2,    21] train loss: 46.5613537
[2,    26] train loss: 49.5604935
[2,    31] train loss: 41.3053633
[2,    36] train loss: 46.1026465
[2,    41] train loss: 43.1602796
[2,    46] train loss: 42.7153746
[2,    51] train loss: 41.8777231
[2,    56] train loss: 44.3936965
[2,    61] train loss: 46.1680775
[2,    66] train loss: 43.7668044
[2,    71] train loss: 40.5722872
[2,    76] train loss: 48

[2,  1136] train loss: 41.9427764
[2,  1141] train loss: 47.9800618
[2,  1146] train loss: 43.0454451
[2,  1151] train loss: 41.7008559
[2,  1156] train loss: 37.5110092
[2,  1161] train loss: 51.0425421
[2,  1166] train loss: 51.6366590
[2,  1171] train loss: 42.7282588
[2,  1176] train loss: 45.1820354
[2,  1181] train loss: 42.7980105
[2,  1186] train loss: 49.5728747
[2,  1191] train loss: 45.3398838
[2,  1196] train loss: 45.4635518
[2,  1201] train loss: 41.2678598
[2,  1206] train loss: 48.7580706
[2,  1211] train loss: 42.9609489
[2,  1216] train loss: 53.5818876
[2,  1221] train loss: 42.1376216
[2,  1226] train loss: 43.9793148
[2,  1231] train loss: 39.5360851
[2,  1236] train loss: 51.2413635
[2,  1241] train loss: 41.2586358
[2,  1246] train loss: 45.2626661
[2,  1250] train loss: 46.6686813
#2 reconstruction test loss: 53.7779426574707
Epoch #2
[3,     1] train loss: 45.6871796
[3,     6] train loss: 42.0641874
[3,    11] train loss: 46.3907439
[3,    16] train loss: 48.4

[3,  1076] train loss: 36.7740949
[3,  1081] train loss: 49.3261038
[3,  1086] train loss: 45.3028380
[3,  1091] train loss: 44.5927842
[3,  1096] train loss: 41.1214701
[3,  1101] train loss: 38.1778431
[3,  1106] train loss: 48.1380889
[3,  1111] train loss: 39.9563179
[3,  1116] train loss: 46.4459565
[3,  1121] train loss: 43.1856333
[3,  1126] train loss: 43.3890788
[3,  1131] train loss: 43.3964310
[3,  1136] train loss: 40.7718379
[3,  1141] train loss: 46.4941559
[3,  1146] train loss: 44.6233908
[3,  1151] train loss: 41.5567907
[3,  1156] train loss: 40.5991548
[3,  1161] train loss: 50.3011430
[3,  1166] train loss: 51.0678641
[3,  1171] train loss: 44.7934386
[3,  1176] train loss: 46.0651824
[3,  1181] train loss: 37.2032331
[3,  1186] train loss: 48.4235744
[3,  1191] train loss: 43.7395356
[3,  1196] train loss: 44.7448902
[3,  1201] train loss: 39.5250289
[3,  1206] train loss: 48.3014952
[3,  1211] train loss: 42.2415053
[3,  1216] train loss: 52.4933713
[3,  1221] tra

[4,  1016] train loss: 41.6949552
[4,  1021] train loss: 47.8410606
[4,  1026] train loss: 39.0603835
[4,  1031] train loss: 48.8113550
[4,  1036] train loss: 42.1962388
[4,  1041] train loss: 40.5206439
[4,  1046] train loss: 44.1024272
[4,  1051] train loss: 37.8639730
[4,  1056] train loss: 41.0222727
[4,  1061] train loss: 42.9422162
[4,  1066] train loss: 51.8279419
[4,  1071] train loss: 46.4971962
[4,  1076] train loss: 36.2296546
[4,  1081] train loss: 46.3321171
[4,  1086] train loss: 42.5683587
[4,  1091] train loss: 43.9338640
[4,  1096] train loss: 40.4726264
[4,  1101] train loss: 39.6230399
[4,  1106] train loss: 44.6584949
[4,  1111] train loss: 43.0724233
[4,  1116] train loss: 44.8650583
[4,  1121] train loss: 42.6599909
[4,  1126] train loss: 46.6231130
[4,  1131] train loss: 40.2346948
[4,  1136] train loss: 41.8968658
[4,  1141] train loss: 48.5369199
[4,  1146] train loss: 45.3133742
[4,  1151] train loss: 42.5101159
[4,  1156] train loss: 38.6296183
[4,  1161] tra

...

[68,   971] train loss: 45.5984446
[68,   976] train loss: 39.0587858
[68,   981] train loss: 37.5467180
[68,   986] train loss: 43.4714260
[68,   991] train loss: 36.2006811
[68,   996] train loss: 39.7211132
[68,  1001] train loss: 36.8257923
[68,  1006] train loss: 37.8520311
[68,  1011] train loss: 36.5243251
[68,  1016] train loss: 39.6124605
[68,  1021] train loss: 44.0017045
[68,  1026] train loss: 36.1120542
[68,  1031] train loss: 40.6612269
[68,  1036] train loss: 43.2747135
[68,  1041] train loss: 38.5267353
[68,  1046] train loss: 42.5778287
[68,  1051] train loss: 34.1301568
[68,  1056] train loss: 42.6984908
[68,  1061] train loss: 40.7505048
[68,  1066] train loss: 44.3819784
[68,  1071] train loss: 42.3529523
[68,  1076] train loss: 36.2446537
[68,  1081] train loss: 40.4622892
[68,  1086] train loss: 40.7469927
[68,  1091] train loss: 43.7952442
[68,  1096] train loss: 38.0783367
[68,  1101] train loss: 41.6036358
[68,  1106] train loss: 43.8137767
[68,  1111] train lo

[69,   876] train loss: 37.4813391
[69,   881] train loss: 39.6520608
[69,   886] train loss: 37.8963960
[69,   891] train loss: 38.2373530
[69,   896] train loss: 35.7381191
[69,   901] train loss: 37.3808142
[69,   906] train loss: 44.1283201
[69,   911] train loss: 39.8602581
[69,   916] train loss: 34.5070607
[69,   921] train loss: 38.4508832
[69,   926] train loss: 39.7730694
[69,   931] train loss: 35.4820302
[69,   936] train loss: 43.4975046
[69,   941] train loss: 41.2825292
[69,   946] train loss: 41.9597880
[69,   951] train loss: 39.4476026
[69,   956] train loss: 43.0883973
[69,   961] train loss: 37.6126792
[69,   966] train loss: 41.1119938
[69,   971] train loss: 43.0531368
[69,   976] train loss: 35.5649910
[69,   981] train loss: 47.1497974
[69,   986] train loss: 41.5544758
[69,   991] train loss: 36.6893342
[69,   996] train loss: 37.7233054
[69,  1001] train loss: 40.1697623
[69,  1006] train loss: 34.7759577
[69,  1011] train loss: 39.3085658
[69,  1016] train lo

[70,   781] train loss: 43.7669983
[70,   786] train loss: 42.9528116
[70,   791] train loss: 39.3037434
[70,   796] train loss: 43.0043125
[70,   801] train loss: 58.3662993
[70,   806] train loss: 39.6810570
[70,   811] train loss: 38.7584496
[70,   816] train loss: 44.6437003
[70,   821] train loss: 44.0844854
[70,   826] train loss: 38.1965968
[70,   831] train loss: 41.8728415
[70,   836] train loss: 36.5105095
[70,   841] train loss: 40.5828641
[70,   846] train loss: 37.5281105
[70,   851] train loss: 39.8826726
[70,   856] train loss: 34.6377989
[70,   861] train loss: 41.4832904
[70,   866] train loss: 44.8667380
[70,   871] train loss: 38.0270812
[70,   876] train loss: 39.5425110
[70,   881] train loss: 38.9588973
[70,   886] train loss: 35.1786346
[70,   891] train loss: 36.9910857
[70,   896] train loss: 34.9949582
[70,   901] train loss: 41.2184188
[70,   906] train loss: 41.6035213
[70,   911] train loss: 40.6090838
[70,   916] train loss: 34.1387024
[70,   921] train lo

[71,   686] train loss: 36.0167561
[71,   691] train loss: 45.1561438
[71,   696] train loss: 36.9012903
[71,   701] train loss: 44.0997798
[71,   706] train loss: 39.6607958
[71,   711] train loss: 37.0344041
[71,   716] train loss: 43.3551852
[71,   721] train loss: 45.2042497
[71,   726] train loss: 40.6109155
[71,   731] train loss: 40.9499302
[71,   736] train loss: 41.3806934
[71,   741] train loss: 33.0939178
[71,   746] train loss: 46.2542686
[71,   751] train loss: 37.3179035
[71,   756] train loss: 38.4696198
[71,   761] train loss: 43.7053261
[71,   766] train loss: 41.2891204
[71,   771] train loss: 36.3660787
[71,   776] train loss: 42.1267262
[71,   781] train loss: 36.9911098
[71,   786] train loss: 40.7199453
[71,   791] train loss: 40.2397830
[71,   796] train loss: 45.9574699
[71,   801] train loss: 55.7009513
[71,   806] train loss: 43.6172676
[71,   811] train loss: 38.5128568
[71,   816] train loss: 47.7426205
[71,   821] train loss: 41.1968327
[71,   826] train lo

[72,   591] train loss: 42.9157785
[72,   596] train loss: 43.4925626
[72,   601] train loss: 35.8229116
[72,   606] train loss: 45.7054132
[72,   611] train loss: 39.3250440
[72,   616] train loss: 41.3029607
[72,   621] train loss: 43.6758760
[72,   626] train loss: 39.2959566
[72,   631] train loss: 41.3029079
[72,   636] train loss: 39.8885492
[72,   641] train loss: 38.7639357
[72,   646] train loss: 38.4817486
[72,   651] train loss: 39.9882170
[72,   656] train loss: 36.6388874
[72,   661] train loss: 47.3192870
[72,   666] train loss: 43.3535913
[72,   671] train loss: 43.6360257
[72,   676] train loss: 41.5380414
[72,   681] train loss: 39.2432505
[72,   686] train loss: 39.9592845
[72,   691] train loss: 41.2232437
[72,   696] train loss: 34.4626509
[72,   701] train loss: 40.6647911
[72,   706] train loss: 38.4385885
[72,   711] train loss: 42.7595272
[72,   716] train loss: 44.5557054
[72,   721] train loss: 42.7951361
[72,   726] train loss: 39.8653049
[72,   731] train lo

Сохраним веса модели

In [8]:
torch.save(dnb_lstm.state_dict(), "models/lstm_dnb_mono_72iter")

Аналогично происходит загрузка модели:

In [9]:
dnb_lstm.load_state_dict(torch.load("models/lstm_dnb_mono_72iter"))

<All keys matched successfully>

In [10]:
batch_drum_train = drum_train[:,:,:].transpose(0,1)
batch_bass_train = bass_train[:,:].transpose(0,1)
with torch.no_grad():
    bass_outputs = dnb_lstm(batch_drum_train)

In [11]:
result = bass_outputs.squeeze().int()
result

tensor([[ 6,  8,  7,  ...,  4,  6,  6],
        [ 6,  5,  8,  ...,  6,  7,  6],
        [ 6,  8,  9,  ...,  4,  7,  8],
        ...,
        [ 6,  9, 18,  ...,  8,  5,  5],
        [ 5,  9, 11,  ...,  3,  3,  6],
        [ 6, 12, 11,  ...,  4,  3,  5]], dtype=torch.int32)

Попробуем сохранить результаты работы сети. На anaconda нет mido, поэтому сохраняем результаты работы просто в массивчик npy... Однако, как альтернатива, его можно поставить чере pip в conda:
https://github.com/mido/mido/issues/198

In [12]:
import mido
from decode_patterns.data_conversion import build_track, DrumMelodyPair, NumpyImage, Converter


# вспомогательная функция для генерации midi
# переводит посл-ть нот в "картинку"
def seq_to_img(bass_seq):
    bass_output = []
    for bass_note in bass_seq:
        bass_row = np.eye(1, 36, bass_note - 1)[0]
        bass_output.append(bass_row)
    bass_output = torch.tensor(bass_output).int().squeeze()
    return bass_output

converter = Converter((16,50))

# batch_drum = torch.cat((drum_train, drum_test, torch.tensor(drum_validation))).transpose(0,1)
# batch_bass = torch.cat((bass_train.int(), bass_test.int(), torch.tensor(bass_validation).int())).transpose(0,1)
def output_midi(batch_drum, batch_bass, folder, output_original=False):
    batch_drum = batch_drum.transpose(0,1)
    batch_bass = batch_bass.int().transpose(0,1)
    with torch.no_grad():
        bass_outputs = dnb_lstm(batch_drum)
        bass_outputs = bass_outputs.squeeze().int()

        for i in range(bass_outputs.size()[1]):
            def output_seq(bass_seq, prefix="sample"):
                bass_output = seq_to_img(bass_seq)

                img_dnb = torch.cat((batch_drum[:,i,:].int(),bass_output), axis=1)
                numpy_pair = NumpyImage(np.array(img_dnb), 120, 1, 1, 36)
                pair = converter.convert_numpy_image_to_pair(numpy_pair)
                mid = build_track(pair, tempo=pair.tempo)
                mid.save(f"{folder}/{prefix}{i+1}.mid")
                
            output_seq(bass_outputs[:,i])
            
            # TODO неплохо вынести генерацию оригинальной музыки в отдельный .py файл... т.е. нужно убрать этот костыль...
            if output_original:
                output_seq(batch_bass[:,i], "orig/original")

Выводим обучающую и валидационную выборку

In [13]:
# # если очень надо послушать тренировчную -- лучше её перезагрузить, потому что она перемешивается
# drum, bass = data_conversion.make_lstm_dataset(height=16, patterns_file="decode_patterns/train.tsv", mono=True)
# drum = torch.tensor(drum, dtype=torch.float)
# bass = torch.tensor(bass, dtype=torch.float)
# output_midi(drum, bass, "midi/lstm_mono/train")
output_midi(drum_validation, bass_validation, "midi/lstm_mono/validation")

По вкусу, выводим тот же результат для кожанных мешков на ассесмент. На самом деле ничем от валидационной выборки не отличается :)

In [14]:
drum_hum, bass_hum = data_conversion.make_lstm_dataset(height=16, patterns_file="decode_patterns/human.tsv", mono=True)
drum_hum = torch.tensor(drum_hum, dtype=torch.float)
bass_hum = torch.tensor(bass_hum, dtype=torch.float)
output_midi(drum_hum, bass_hum, "midi/lstm_mono/human")